<a href="https://colab.research.google.com/github/dimiphoton/imdb-project-genre-prediction-recommendation-deployment/blob/main/Copie_de_mvp_genre_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 37.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 69.1 MB/s eta 0:00:00


In [4]:
import pandas as pd
import numpy as np
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
import spacy
from transformers import BertTokenizer, BertForSequenceClassification, BertConfig, AdamW

In [5]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [6]:
file_path = "/content/drive/MyDrive/Colab Notebooks/imdb/processed_data.csv"
df = pd.read_csv(file_path)

In [7]:
df.head()

,original_title,genre_list,overview
0,Toy Story,"['Animation', 'Comedy', 'Family']","Led by Woody, Andy's toys live happily in his ..."
1,Jumanji,"['Adventure', 'Fantasy', 'Family']",When siblings Judy and Peter discover an encha...
2,Grumpier Old Men,"['Romance', 'Comedy']",A family wedding reignites the ancient feud be...
3,Waiting to Exhale,"['Comedy', 'Drama', 'Romance']","Cheated on, mistreated and stepped on, the wom..."
4,Father of the Bride Part II,['Comedy'],Just when George Banks has recovered from his ...


In [8]:
import os

google_drive_folder = "/content/drive/MyDrive/Colab Notebooks/imdb"
print(os.listdir(google_drive_folder))


['processed_data.csv', 'bert_base_uncased', 'data_afterENR.csv', 'data_afterEmbedding.csv', 'finetuned_bert', 'fine-tuned-bert']


In [37]:
# Instantiate the fine-tuned BERT model
num_labels = len(mlb.classes_)
output_path = "/content/drive/MyDrive/Colab Notebooks/imdb/bert_base_uncased"
base_model = BertModel.from_pretrained(output_path)
model = CustomBertModel(base_model, num_labels)


In [7]:
# Assuming your Google Drive is already mounted
output_path = "/content/drive/MyDrive/Colab Notebooks/imdb/fine-tuned-bert"

# Create the output folder if it doesn't exist
os.makedirs(output_path, exist_ok=True)

from transformers import BertTokenizer, BertModel

model_name = 'fine-tuned-bert'

# Download the BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)
# Save the BERT model and tokenizer to the output folder
model.save_pretrained(output_path)
tokenizer.save_pretrained(output_path)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


('/content/drive/MyDrive/Colab Notebooks/imdb/bert_base_uncased/tokenizer_config.json',
 '/content/drive/MyDrive/Colab Notebooks/imdb/bert_base_uncased/special_tokens_map.json',
 '/content/drive/MyDrive/Colab Notebooks/imdb/bert_base_uncased/vocab.txt',
 '/content/drive/MyDrive/Colab Notebooks/imdb/bert_base_uncased/added_tokens.json')

#  Encoding the target labels (genres)

In [8]:
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer()
df['genre_list_encoded'] = list(mlb.fit_transform(df['genre_list']))


In [9]:
df.head()

,original_title,genre_list,overview,genre_list_encoded
0,Toy Story,"['Animation', 'Comedy', 'Family']","Led by Woody, Andy's toys live happily in his ...","[1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
1,Jumanji,"['Adventure', 'Fantasy', 'Family']",When siblings Judy and Peter discover an encha...,"[1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
2,Grumpier Old Men,"['Romance', 'Comedy']",A family wedding reignites the ancient feud be...,"[1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, ..."
3,Waiting to Exhale,"['Comedy', 'Drama', 'Romance']","Cheated on, mistreated and stepped on, the wom...","[1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, ..."
4,Father of the Bride Part II,['Comedy'],Just when George Banks has recovered from his ...,"[0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


# Extracting named entities from the movie summaries

In [10]:
import torch

if torch.cuda.is_available():
    print("GPU is available.")
    print("GPU name:", torch.cuda.get_device_name(0))
else:
    print("GPU is not available.")

GPU is available.
GPU name: Tesla T4


In [11]:
nlp = spacy.load('en_core_web_sm')

def extract_named_entities(text):
    doc = nlp(text)
    return [ent.text for ent in doc.ents]

df['named_entities'] = df['overview'].apply(extract_named_entities)


In [17]:
import os

output_folder = "/content/drive/MyDrive/Colab Notebooks/imdb/"
output_file_name = "data_afterENR.csv"

# Create the output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Save the DataFrame as a CSV file
output_path = os.path.join(output_folder, output_file_name)
df.to_csv(output_path, index=False)

# Embedding movie summaries using BERT

Now, we'll use BERT to create embeddings for the movie summaries. First, we'll import the necessary libraries and load the BERT model.

### the loop method (heavy)

In [22]:
def get_bert_embeddings(texts, tokenizer, model, batch_size=64, device='cuda'):
    model.to(device)
    model.eval()

    embeddings = []
    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i+batch_size]

        input_ids = tokenizer.batch_encode_plus(
            batch_texts,
            return_tensors='pt',
            padding=True,
            truncation=True,
            max_length=128
        ).to(device)

        with torch.no_grad():
            outputs = model(**input_ids)
        
        batch_embeddings = outputs[1].cpu().numpy()
        embeddings.extend(batch_embeddings)
    
    return np.array(embeddings)

# Get embeddings for the entire dataset
overview_embeddings = get_bert_embeddings(df['overview'].tolist(), tokenizer, model)
df['overview_embedding'] = list(overview_embeddings)


In [24]:
output_folder = "/content/drive/MyDrive/Colab Notebooks/imdb/"
output_file_name = "data_afterEmbedding.csv"

# Create the output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Save the DataFrame as a CSV file
output_path = os.path.join(output_folder, output_file_name)
df.to_csv(output_path, index=False)

# Fine-tuning the BERT model
First, we need to create  PyTorch Dataset and DataLoader for both the training and validation sets.

In [31]:
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

class MovieDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length):
        self.dataframe = dataframe
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        row = self.dataframe.iloc[idx]
        encoding = self.tokenizer.encode_plus(
            row['overview'],
            add_special_tokens=True,
            max_length=self.max_length,
            return_token_type_ids=False,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt',
            truncation=True
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'targets': torch.tensor(row['genre_list_encoded'], dtype=torch.float)
        }

# Split the data
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

# Create DataLoaders
train_dataset = MovieDataset(train_df, tokenizer, max_length=128)
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)

val_dataset = MovieDataset(val_df, tokenizer, max_length=128)
val_dataloader = DataLoader(val_dataset, batch_size=16, shuffle=False)



Next, we'll create the PyTorch Dataset and DataLoader for both the training and validation sets.

In [38]:
from transformers import BertTokenizer, BertModel, AdamW
from torch.optim import lr_scheduler
import torch.nn as nn
import torch

# Define a custom BERT model with a classification head
class CustomBertModel(nn.Module):
    def __init__(self, bert_model, num_labels):
        super().__init__()
        self.bert = bert_model
        self.classifier = nn.Linear(bert_model.config.hidden_size, num_labels)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        logits = self.classifier(outputs.pooler_output)
        return logits

# Load tokenizer and base model
output_path = "/content/drive/MyDrive/Colab Notebooks/imdb/bert_base_uncased"
tokenizer = BertTokenizer.from_pretrained(output_path)
base_model = BertModel.from_pretrained(output_path)

# Instantiate the custom BERT model
num_labels = len(mlb.classes_)
model = CustomBertModel(base_model, num_labels)

# Use GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Set up optimizer, criterion, and scheduler
optimizer = AdamW(model.parameters(), lr=2e-5, correct_bias=False)
criterion = nn.BCEWithLogitsLoss()
scheduler = lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

# Training loop
epochs = 5
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for batch in train_dataloader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        targets = batch['targets'].to(device)

        optimizer.zero_grad()
        logits = model(input_ids=input_ids, attention_mask=attention_mask)
        loss = criterion(logits, targets)
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        scheduler.step()

        total_loss += loss.item()
    print(f"Epoch {epoch+1} Loss: {total_loss / len(train_dataloader)}")


Epoch 1 Loss: 0.5887986393914523
Epoch 2 Loss: 0.5887321094549532
Epoch 3 Loss: 0.5887145969552601
Epoch 4 Loss: 0.5887017638295932
Epoch 5 Loss: 0.5887291106721453


In [40]:
# Create a folder to save the model and tokenizer
save_path = "/content/drive/MyDrive/Colab Notebooks/imdb/fine-tuned-bert"
os.makedirs(save_path, exist_ok=True)

# Save the trained model
torch.save(model.state_dict(), os.path.join(save_path, "model.pth"))

# Save the tokenizer
tokenizer.save_pretrained(save_path)


('/content/drive/MyDrive/Colab Notebooks/imdb/fine-tuned-bert/tokenizer_config.json',
 '/content/drive/MyDrive/Colab Notebooks/imdb/fine-tuned-bert/special_tokens_map.json',
 '/content/drive/MyDrive/Colab Notebooks/imdb/fine-tuned-bert/vocab.txt',
 '/content/drive/MyDrive/Colab Notebooks/imdb/fine-tuned-bert/added_tokens.json')

In [2]:
import torch
from transformers import BertModel, BertTokenizer

# Set the save path where the model and tokenizer were saved
save_path = "/content/drive/MyDrive/Colab Notebooks/imdb/finetuned_bert"

# Load the tokenizer
tokenizer = BertTokenizer.from_pretrained(save_path)

# Load the base model
base_model = BertModel.from_pretrained(save_path)

# Load the model's state dictionary (learned weights)
model_state_dict = torch.load(os.path.join(save_path, "model.pth"))

# Instantiate the CustomBertModel with the number of labels (change this to the actual number of labels in your dataset)
num_labels = len(mlb.classes_)
model = CustomBertModel(base_model, num_labels)

# Load the state dictionary into the CustomBertModel
model.load_state_dict(model_state_dict)

# Set the model to the appropriate device (GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


ModuleNotFoundError: ignored

# Making predictions

After training the model, we'll create a function to make predictions on new summaries.

In [59]:
print(len(mlb.classes_))


8


In [56]:
def predict_genre(text, model, tokenizer, mlb, threshold=0.5):
    input_ids = tokenizer.encode(text, return_tensors='pt').to(device)
    attention_mask = (input_ids != tokenizer.pad_token_id).unsqueeze(-1).to(device)
    with torch.no_grad():
        logits = model(input_ids, attention_mask)
    probabilities = torch.sigmoid(logits)
    predictions = (probabilities > threshold).cpu().numpy()
    genres = mlb.inverse_transform(predictions)
    return [''.join(genre) for genre in genres]




In [1]:
new_summary = "A group of intergalactic criminals is forced to work together to stop a fanatical warrior from taking control of the universe."
predicted_genres = predict_genre(new_summary, model, tokenizer, mlb)
print(predicted_genres)




NameError: ignored

# in a nutshell: how is the data vectorized?

## Vectorization of input features (X)

The input features are the movie overviews. We use the BERT tokenizer to tokenize the movie overviews and then create an embedding for each overview. The BERT tokenizer converts each overview into a sequence of tokens, which are then represented by their corresponding IDs. The resulting vector has a fixed length (in this case, 512) and is padded or truncated as needed.

Here's how the tokenization and embedding process works:

In [49]:
from transformers import BertTokenizer

#tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def tokenize_overview(overview):
    return tokenizer.encode_plus(
        overview,
        max_length=512,
        truncation=True,
        padding='max_length',
        return_attention_mask=True,
        return_tensors='pt',
    )

overview = "Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene."
tokenized_overview = tokenize_overview(overview)

print(f"Tokenized overview: {tokenized_overview['input_ids']}")
print(f"Attention mask: {tokenized_overview['attention_mask']}")


Tokenized overview: tensor([[  101,  2419,  2011, 13703,  1010,  5557,  1005,  1055, 10899,  2444,
         11361,  1999,  2010,  2282,  2127,  5557,  1005,  1055,  5798,  7545,
         12610,  2422, 29100,  3031,  1996,  3496,  1012,   102,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,   

## Vectorization of target labels (y)
The target labels are the movie genres. Since a movie can have multiple genres, we use one-hot encoding to represent the genre labels. One-hot encoding creates a binary vector for each genre, where the element corresponding to the genre's index is set to 1, and all other elements are set to 0.

Here's how the one-hot encoding process works:

In [50]:
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer()

# Let's assume we have the following genre list
genre_list = [['Animation', 'Comedy', 'Family'],
              ['Adventure', 'Fantasy', 'Action'],
              ['Science Fiction', 'Action', 'Thriller']]

# Fit the MultiLabelBinarizer to the genre list and transform it
genre_encoded = mlb.fit_transform(genre_list)

print(f"Encoded genre list: {genre_encoded}")
print(f"Genre classes: {mlb.classes_}")


Encoded genre list: [[0 0 1 1 1 0 0 0]
 [1 1 0 0 0 1 0 0]
 [1 0 0 0 0 0 1 1]]
Genre classes: ['Action' 'Adventure' 'Animation' 'Comedy' 'Family' 'Fantasy'
 'Science Fiction' 'Thriller']


# probabilistics recommender with a bunching parameter

In [ ]:
import heapq
from scipy.spatial.distance import cosine

In [ ]:
def compute_similarity(overview1, overview2):
    tokenized_overview1 = tokenize_overview(overview1)['input_ids'].to(device)
    tokenized_overview2 = tokenize_overview(overview2)['input_ids'].to(device)
    
    with torch.no_grad():
        embedding1 = model.bert(tokenized_overview1)[1].cpu().numpy()
        embedding2 = model.bert(tokenized_overview2)[1].cpu().numpy()
        
    return 1 - cosine(embedding1, embedding2)


In [ ]:
def recommend_movies(movie_names, bunching, df):
    movie_indices = df[df['original_title'].isin(movie_names)].index.tolist()
    
    if len(movie_indices) != len(movie_names):
        print("Some movies were not found in the dataset.")
        return
    
    similarity_scores = []
    
    for index, row in df.iterrows():
        if index not in movie_indices:
            total_similarity = sum([compute_similarity(row['overview'], df.loc[i]['overview']) for i in movie_indices])
            adjusted_similarity = total_similarity / len(movie_indices)
            similarity_scores.append((index, adjusted_similarity))
    
    if bunching > 0:
        # Recommend similar movies
        top_indices = heapq.nlargest(5, similarity_scores, key=lambda x: x[1])
    else:
        # Recommend dissimilar movies
        top_indices = heapq.nsmallest(5, similarity_scores, key=lambda x: x[1])
    
    recommended_movies = df.loc[[index for index, _ in top_indices]]
    
    return recommended_movies


In [ ]:
# Example
movie_names = ['Toy Story', 'Jumanji', 'Heat', 'GoldenEye', 'The American President']
bunching = 0.5

recommended_movies = recommend_movies(movie_names, bunching, df)
print(recommended_movies[['original_title', 'overview', 'genre_list']])
